In [175]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv('netflix.csv')
df.tail(1000)

In [ ]:
df.shape

In [ ]:
#visualise the close price data
plt.figure(figsize=(16,8))
plt.title('Netflix Close Price Data')
plt.xlabel('Days')
plt.ylabel('close price')
plt.plot(df['Close'][4000:])
plt.show()

In [ ]:
df_training = df[df['Date'] < '2018-01-01']
df_training

In [ ]:
df_testing = df[df['Date'] >= '2018-01-01']
df_testing

In [ ]:
Adj_training = df_training.drop(['Date','Adj Close'],1)
Adj_training.head()

In [172]:
Adj_testing = df_testing.drop(['Date','Adj Close'],1)
Adj_testing.shape

(951, 5)

In [ ]:
scaler = MinMaxScaler()
Adj_training = scaler.fit_transform(Adj_training)
Adj_testing = scaler.fit_transform(Adj_testing)
Adj_training

In [142]:
X_train = []
y_train = []
X_test = []
y_test = []

In [ ]:
Adj_training.shape

In [144]:
for i in range(60, Adj_training.shape[0]):
  X_train.append(Adj_training[i-60:i])
  y_train.append(Adj_training[i, 0])

In [145]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
print(X_train.shape)

In [147]:
for i in range(60, Adj_testing.shape[0]):
  X_test.append(Adj_testing[i-60:i])
  y_test.append(Adj_testing[i, 0])

In [148]:
X_test, y_test = np.array(X_test), np.array(y_test)

In [149]:
model = Sequential()
layer = Dropout(0.2)
model.add(LSTM(units = 50, activation='relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
model.add(layer)
model.add(LSTM(units = 50, activation='relu', return_sequences = True))
model.add(layer)
model.add(LSTM(units = 50, activation='relu', return_sequences = True))
model.add(layer)
model.add(LSTM(units = 50, activation='relu'))
model.add(layer)

model.add(Dense(units = 1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'mean_squared_error')
model.fit(X_train,y_train,epochs = 20,batch_size = 32)

In [ ]:
X_test.shape


In [ ]:
y_predict = model.predict(X_test)
y_predict

In [ ]:
scale = scaler.scale_[0]
scale

In [ ]:
y_predict = y_predict / scale
y_test = y_test / scale
y_test.shape

In [ ]:
##visualise
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Days')
plt.ylabel('predict price')
plt.plot(y_test, color = 'red', label = 'Real price')
plt.plot(y_predict, color = 'green', label = 'Predicted price')
plt.legend()
plt.show()

In [ ]:

print('MSE:')
print(mean_squared_error(y_test*scale, y_predict*scale)/891*638.7271185815334)